# Graphite-шаблоны в InfluxDB


Расскажите про язык graphite шаблонов в influxdb. Покажите работающий пример. В идеале — набросать на python небольшой notebook, в котором вы берёте данные в формате графит и преобразуете их в формат межермент-тэг-поле (как это принято в инфлюксе). Используйте разнообразные шаблоны. (5 минут, 10 с ноутбуком)

*Note: для работы скриптов у вас должен быть установлен и настроен influxdb, конфиг для работы с графитом ниже* 

### Graphite

#### Metrics

Метрики однозначно задаются своим именем (последовательность строк разделенных точкой) и (опционально) набором тегов.

Структура:  
`path.to.value;tag1=val1;tag2=val2;...`

Например, `disk.used;datacenter=dc1;rack=a1;server=web01`

#### Messages

Формат для сообщений: `metric_path value timestamp`
- `metric_path` - путь до метрики (см. выше)
- `value` - значение, которое нужно положить
- `timestamp` - временная метка. Если присвоить `-1`, **Carbon-cache** сам определит, что поставить

Например, `test.bash.stats 10 -1`

### InfluxDB

#### Временные ряды (aka time series)

Основная единица хранения информации - временные ряды. Ряды состоят из точек (конкретных измерений данного ряда).

Структура:  
`<measurement>[,<tag-key>=<tag-value>...] <field-key>=<field-value>[,<field2-key>=<field2-value>...] [timestamp]`

- `measurement` - что хотим измерить
- `tags` - теги 
- `fields` - значения измерения (может быть несколько)
- `timestamp` - временная метка (если не указана, используется текущий локальный timestamp)

Например, `temperature,machine=unit42,type=assembly external=25,internal=37 1434067467000000000`

### Graphite шаблоны в InfluxDB

В конфиге можно указать, по какому шаблону мы хотим получать graphite-данные.

Есть ключевые слова:  
 - `measurement` - для, собственно, measurement'a в инфлюксе.  
 - `field` - для значения. Если не указать, будет назван `value`.

Остальные слова являются именами для тегов.

##### Пример
 - Шаблон: `.host.resource.measurement*`  
 - Метрика: `servers.localhost.cpu.loadavg.10 555` => `loadavg.10,host=localhost,resource=cpu value=555`

`measurement` может быть прописан несколько раз, тогда сматчившиеся подстроки сконкатенируются с разделителем (по дефолту `.`, можно указать в конфиге). C `field` такое не прокатит.

Можно добавлять свои теги к шаблону.

#### Фильтры

Допустим, мы добавили несколько шаблонов для матча. Чтобы определять, какая точка должна пойти в какой шаблон, перед ним надо добавить *фильтр*.

Можно задавать дефолтные шаблоны для парсинга тех значений, которые не подошли ни под один фильтр.  
Можно добавить дефолтные теги, которые будут добавляться к каждой записи.

### Пример конфига
```
 [[graphite]]
  enabled = true
  bind-address = ":2103"
   
  separator = "_"

  tags = ["region=moscow", "answer=42"]
  
   templates = [
     "servers.* .host.resource.measurement.field.measurement* extra_tag=extra_value",
     "stats.* .host.name.measurement*",
     ".measurement*"
   ]
```

In [1]:
import socket
import time
import random
import re
import requests

#### Скрипт, генeрирующий и посылающий graphite-данные в carbon, которые затем кладутся в InfluxDB (carbon-send.py)

In [ ]:
CARBON_SERVER = 'localhost'
CARBON_PORT = 2103 # write your carbon port here (default 2003)

metric_path = "servers.local.cpu.prefix.avg.middle.suffix"
value = 0 
sock = socket.socket()
sock.connect((CARBON_SERVER, CARBON_PORT))
while True:
    value = random.uniform(-100, 100) 
    time.sleep(1)
    message = '%s %s\n' % (metric_path, value)
    sock.sendall(message.encode('utf-8'))
sock.close()

#### Ручной парсер graphite -> influx метрик

In [4]:
class Template:
    def __init__(self):
        self.filter = None
        self.matcher = None
        self.default = []
        
    def match(self, metric):
        if self.filter is None:
            return True
        return self.filter.match(metric) is not None
    
    def parse(self, metrics, sep, default_tags, value):
        measurement = ''
        field_name = 'value'
        tags = {}
        for tag in default_tags:
            name, val = tag.split('=')
            tags[name] = val
        for tag in self.default:
            name, val = tag.split('=')
            tags[name] = val
        for i in range(len(metrics)):
            name = self.matcher[i]
            val = metrics[i]
            if '' == name:
                continue
            if 'measurement' == name:
                measurement += sep + val
            elif 'measurement*' == name:
                measurement += sep + sep.join(list(metrics[k] for k in range(i, len(metrics))))
                break
            elif 'field' == name:
                field_name = val
            elif 'field*' == name:
                field_name = sep.join(list(metrics[k] for k in range(i, len(metrics))))
                break
            else:
                tags[name] = val
        
        answer = measurement[1:]
        answer += ',' + ','.join(['='.join(k) for k in tags.items()])
        answer += ' ' + field_name + '=' + value
        return answer
    
class MetricParser:
    def __init__(self, sep='.', default_tags=None):
        self.sep = sep
        self.default_tags = default_tags.split(',')
        self.templates = []
        
    def add(self, template):
        tokens = template.split()
        t = Template()
        filter_index = matcher_index = default_index = 3
        if len(tokens) == 3:
            filter_index = 0
            matcher_index = 1
            default_index = -1
        elif len(tokens) == 2:
            if '=' in tokens[-1]:
                matcher_index = 0
                default_index = -1
            else:
                filter_index = 0
                matcher_index = 1
        else:
            matcher_index = 0
        t.matcher = tokens[matcher_index].split('.')
        if filter_index != 3:
            t.filter = re.compile(tokens[filter_index].replace('.', '\.').replace('*', '[^\.]*'))
        if default_index != 3:
            t.default = tokens[default_index].split(',')
        self.templates.append(t)
        return self
    
    def parse(self, metric):
        try:
            metric, value = metric.split(' ')
            for t in self.templates:
                if t.match(metric):
                    return t.parse(metric.split('.'), self.sep, self.default_tags, value)
        except ValueError:
            print("Invalid format")

In [5]:
parser = MetricParser(sep='_', default_tags='region=moscow,answer=42')
parser = parser.add('servers.* .host.resource.measurement.field.measurement* extra_tag=extra_value')\
.add('stats.* .host.name.measurement*')\
.add('failures.*.db1 ...type.measurement.field')\
.add('.measurement*')

print(parser.parse('servers.local.cpu.prefix.avg.middle.suffix 10'))
print()
print(parser.parse('aaa.bbb.ccc 24'))
print()
print(parser.parse('stats.local.stat1.a.b.c.d.e.f 432'))
print()
print(parser.parse('failures.russia.db1.hardware.loss.dollars 25300'))

prefix_middle_suffix,region=moscow,answer=42,extra_tag=extra_value,host=local,resource=cpu avg=10

bbb_ccc,region=moscow,answer=42 value=24

a_b_c_d_e_f,region=moscow,answer=42,host=local,name=stat1 value=432

loss,region=moscow,answer=42,type=hardware dollars=25300


#### Скрипт для ручного преобразования graphite-данных и дальнейшей вставки в базу (parser-send.py)

In [ ]:
address = 'http://localhost:8086/write?db=graphite'

while True:
    value = random.uniform(-100, 100)
    time.sleep(1)
    data = parser.parse('servers.local.gpu.prefix.avg.middle.suffix {0}').format(value)
    r = requests.post(address, data=data)